### Imports and Setup

In [21]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

### Set Library Directory and List CSVs

In [22]:
# Define the folder containing section CSVs
library_folder = "SteelLibrary/Members/Main"
# List all relevant CSV files
csv_files = [f for f in os.listdir(library_folder) if f.endswith(".csv")]
section_types = sorted(set(f[:3] for f in csv_files))  # e.g., "PFC", "UB", "RHS"

### 🏗️ Section Library Setup

In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Setup ---
library_folder = "SteelLibrary/Members/Main"
csv_files = [f for f in os.listdir(library_folder) if f.endswith(".csv")]
section_types = sorted(set(f[:3] for f in csv_files))

section_type_dropdown = widgets.Dropdown(
    options=section_types, description="Section Type:", layout=widgets.Layout(width="250px")
)
series_dropdown = widgets.Dropdown(options=[], description="Series:", layout=widgets.Layout(width="250px"))
member_dropdown = widgets.Dropdown(options=[], description="Member:", layout=widgets.Layout(width="350px"))
material_output = widgets.Output()
geometry_output = widgets.Output()
current_df = pd.DataFrame()

section_props = {}

# --- Dropdown Behavior ---
def get_grade_from_series(series_name):
    try:
        return int(''.join(filter(str.isdigit, series_name))) * 10
    except:
        return "Unknown"

def update_series(change):
    section_type = change['new']
    options = sorted([f.replace(".csv", "") for f in csv_files if f.startswith(section_type)])
    series_dropdown.options = options
    member_dropdown.options = []
    material_output.clear_output()
    geometry_output.clear_output()

def update_member_list(change):
    global current_df
    full_path = os.path.join(library_folder, f"{change['new']}.csv")
    try:
        df = pd.read_csv(full_path, skiprows=[1])
        current_df = df
        member_dropdown.options = df["Description"].dropna().tolist()
    except Exception as e:
        current_df = pd.DataFrame()
        member_dropdown.options = []
        with material_output:
            print(f"❌ Error reading file: {e}")

def display_properties(change):
    material_output.clear_output()
    geometry_output.clear_output()
    member = change['new']

    if current_df.empty or member not in current_df["Description"].values:
        return

    row = current_df[current_df["Description"] == member].iloc[0]
    series = series_dropdown.value
    grade = get_grade_from_series(series)
    E = 200_000

    # --- Define hardcoded scaling factors ---
    scaling = {
        "Ix": 1e6,     # mm⁴
        "Zx": 1e3,     # mm³
        "Sx": 1e3,     # mm³
        "Iy": 1e6,     # mm⁴
        "Zy": 1e3,     # mm³
        "Sy": 1e3,     # mm³
        "J": 1e3,      # mm⁴
        "Iw": 1e9,     # mm⁶
        "Zex": 1e3,    # mm³
        "Zey": 1e3     # mm³
    }

    section_props['E'] = E * 1e6  # Store E (MPa → Pa)

    with material_output:
        print(f"✅ Selected: {member}")
        print(f"🟦 Grade: {grade}")
        print(f"📐 E: {E} MPa")

    # Only display selected geometric properties
    display_geom_props = ["d", "bf", "tf", "tw", "Ix", "Zx", "Sx", "rx"]
    filtered = row[display_geom_props].to_frame(name='Value')
    with geometry_output:
        display(filtered)


    # --- Loop over all columns and define scaled or raw globals ---
    for col in row.index:
        raw_val = row[col]
        if pd.notna(raw_val) and (
            isinstance(raw_val, (int, float)) or 
            (isinstance(raw_val, str) and raw_val.replace('.', '', 1).isdigit())
        ):
            try:
                val = float(raw_val) * scaling.get(col, 1)
                globals()[col] = val
                section_props[col] = val
                print(f"📌 {col} = {val}")
            except Exception as e:
                print(f"⚠️ Could not convert: {col} = {raw_val} ({e})")
        else:
            print(f"❌ Skipped: {col} (non-numeric or missing)")



# --- Event Hooks ---
section_type_dropdown.observe(update_series, names="value")
series_dropdown.observe(update_member_list, names="value")
member_dropdown.observe(display_properties, names="value")

# --- Display UI ---
section_ui = widgets.VBox([
    section_type_dropdown,
    series_dropdown,
    member_dropdown,
    widgets.HTML("<b>Material Properties</b>"),
    material_output,
    widgets.HTML("<b>Geometric Properties</b>"),
    geometry_output
])
display(section_ui)
update_series({'new': section_type_dropdown.value})


In [25]:
Ix

1230000.0

# Imports and Setup